# Final Project - THE BATTLE OF NEIGHBOURHOOD
## Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem

This project aims to select or to suggest the best place in <b>India</b> for the targeted people to get settled. 

The report will be targeted to people who are looking to <b>settle in India.</b>

For settlement, or to find a neighbourhood to hunt for an apartment, we will be focusing on the <b>four metropolitan cities</b> only - namely, <b>Delhi, Mumbai, Chennai and Kolkata.</b>

We will explore the neighbourhood of all four cities and finally cluster the neighbourhoods using k-mean clustering.

We will suggest different cities on basis of their preferences and the atmosphere.